# Lab 1: Single-qubit and multi-qubit states and Quantum Fourier Transform

In this lab, you will learn how to write `Qiskit` code and investigate single-qubit and multi-qubit states.

If you have not used Jupyter notebooks before, take a look at the following video to quickly get started.
- https://www.youtube.com/watch?v=jZ952vChhuI

Remember, to run a cell in Jupyter notebooks, you press `Shift` + `Return/Enter` on your keyboard.

## Installing necessary packages

Before we begin, you will need to install some prerequisites into your environment. Run the cell below to complete these installations. At the end, the cell outputs will be cleared.

In [ ]:
!pip install qiskit
!pip install qiskit_aer
!pip install qiskit_algorithms
!pip install pylatexenc
!pip install matplotlib
!pip install seaborn
from IPython.display import clear_output
clear_output()

# Quantum states

## Single-qubit states
We learned that single qubit states can be written down generally as
$$\alpha_0 \vert0\rangle + \alpha_1 \vert1\rangle.$$

Using what we know about quantum states, this can be rewritten (why?) as:

$$\sqrt{1-p}\vert0\rangle + e^{i\phi}\sqrt{p}\vert1\rangle$$

Here, $p$ is the probability that a measurement of the state in the computational basis $\{\vert0\rangle, \vert1\rangle\}$ will have the outcome $1$, and $\phi$ is the phase between the two computational basis states.

Single-qubit gates can then be used to manipulate this quantum state by changing either $p$, $\phi$, or both.

Let's begin by creating a single-qubit quantum circuit. We can do this in `Qiskit` using the following:

In [ ]:
from qiskit import QuantumCircuit
from qiskit import *
import numpy as np

mycircuit = QuantumCircuit(1)
mycircuit.draw('mpl')

The above quantum circuit does not contain any gates. Therefore, if you start in any state, say $\vert0\rangle$, applying this circuit to your state doesn't change the state.

To see this clearly, let's create the statevector $\vert0\rangle$. In `Qiskit`, you can do this using the following:

In [ ]:
from qiskit.quantum_info import Statevector

sv = Statevector.from_label('0')

You can see what's contained in the object `sv`:

In [ ]:
sv

The vector itself can be found by writing

In [ ]:
sv.data

This matches what we learned in class. Recall that $$\vert0\rangle = \begin{bmatrix}1\\0\end{bmatrix}$$

We can now apply the quantum circuit `mycircuit` to this state by using the following:

In [ ]:
new_sv = sv.evolve(mycircuit)

Once again, you can look at the new statevector by writing

In [ ]:
new_sv

The statevector hasn't changed. You can compute the projection of `new_sv` onto `sv` by writing

In [ ]:
from qiskit.quantum_info import state_fidelity

state_fidelity(sv, new_sv)

This computes a measure of similarity between states. For the type of states that we see here, it is the square of the absolute value of the inner product. The projection of `new_sv` onto `sv` is 1, indicating that the two states are identical. You can visualize this state using the `qsphere` by writing

More information about fidelity: https://en.wikipedia.org/wiki/Fidelity_of_quantum_states

In [ ]:
from qiskit.visualization import plot_state_qsphere
plot_state_qsphere(sv.data)

- The `1 minute Qiskit` episode entitled `What is the qsphere?` succinctly describes the Qsphere visualization tool that we used in this lab. You can find it here: https://youtu.be/4SoK2h4a7us


# Bloch Sphere


$$\cos {\frac {\theta }{2}} \vert0\rangle + e^{i\phi}\sin {\frac {\theta }{2}}\vert1\rangle$$

https://en.wikipedia.org/wiki/Bloch_sphere

In [ ]:
from qiskit.visualization import *
plot_bloch_vector([0,0,1])

In [ ]:
sv

Applying an $X$ gate flips the qubit from the state $\vert0\rangle$ to the state $\vert1\rangle$. To see this clearly, we will first create a single-qubit quantum circuit with the $X$ gate.

In [ ]:
mycircuit = QuantumCircuit(1)
mycircuit.x(0)

mycircuit.draw('mpl')

Now, we can apply this circuit onto our state by writing

In [ ]:
sv = Statevector.from_label('0')
new_sv = sv.evolve(mycircuit)
new_sv

The statevector now corresponds to that of the state $\vert1\rangle$. Recall that

$$\vert1\rangle = \begin{bmatrix}0\\1\end{bmatrix}$$

Now, the projection of `new_sv` onto `sv` is

In [ ]:
state_fidelity(new_sv, sv)

This is not surprising. Recall that the states $\vert0\rangle$ and $\vert1\rangle$ are orthogonal. Therefore, $\langle0\vert1\rangle = 0$. The state can be shown on the `qsphere` by writing

In [ ]:
plot_state_qsphere(new_sv.data)

Similarly, we can create the state $$\frac{1}{\sqrt{2}}\left(\vert0\rangle + \vert1\rangle\right)$$
by applying a Hadamard gate. Here is how we can create the state and visualize it in `Qiskit`:

In [ ]:
sv = Statevector.from_label('0')
mycircuit = QuantumCircuit(1)
mycircuit.h(0)
mycircuit.draw('mpl')

In [ ]:
new_sv = sv.evolve(mycircuit)
print(new_sv)
plot_state_qsphere(new_sv.data)

We can see that the state has equal components of $\vert0\rangle$ and $\vert1\rangle$. The size of the circle is proportional to the probability of measuring each basis state in the statevector. As a result, we can see that the size of the circles is half of the size of the circles in our previous visualizations.

Recall from lecture that we can also create other superpositions with different phase. Let's create $$\frac{1}{\sqrt{2}}\left(\vert0\rangle - \vert1\rangle\right)$$ which can be done by applying the Hadamard gate on the state $\vert1\rangle$.

In [ ]:
sv = Statevector.from_label('1')
mycircuit = QuantumCircuit(1)
mycircuit.h(0)

new_sv = sv.evolve(mycircuit)
print(new_sv)
plot_state_qsphere(new_sv.data)

This time, the bottom circle, corresponding to the basis state $\vert1\rangle$ has a different color corresponding to the phase of $\phi = \pi$. This is because the coefficient of $\vert1\rangle$ in the state $$\frac{1}{\sqrt{2}}\left(\vert0\rangle - \vert1\rangle\right)$$ is $-1$, which is equal to $e^{i\pi}$.

Other phases can also be created by applying different gates. The $T$ and $S$ gates apply phases of $+\pi/4$ and $+\pi/2$, respectively.

## Multi-qubit states
We can also explore multi-qubit gates in `Qiskit`. We discussed states of this form:
$$\frac{1}{\sqrt{2}}\left(\vert00\rangle + \vert11\rangle\right).$$
This is called a `Bell state`. We have seen how it can be generated using quantum gates. We will demonstrate below how to create this state from the state $\vert00\rangle$. We will start by visualizing the state $\vert00\rangle$ using the same procedure:

In [ ]:
from qiskit.visualization import plot_bloch_multivector

sv = Statevector.from_label('00')
plot_state_qsphere(sv.data)



Next, we use the Hadamard gate described above, along with a controlled-X gate, to create the Bell state.

In [ ]:
plot_bloch_multivector(sv)

In [ ]:
mycircuit = QuantumCircuit(2)
mycircuit.h(0)
mycircuit.cx(0,1)
mycircuit.draw('mpl')

Note: Qiskit CX is different: https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html

The result of this quantum circuit on the state $\vert00\rangle$ is found by writing

In [ ]:
new_sv = sv.evolve(mycircuit)
print(new_sv)
plot_state_qsphere(new_sv.data)

Following entanglement, it is no longer possible to treat the two qubits individually, and they must be considered to be one system.

To see this clearly, we can see what would happen if we measured the Bell state above 1000 times.

In [ ]:
counts = new_sv.sample_counts(shots=1000)

from qiskit.visualization import plot_histogram
plot_histogram(counts)

All measurements give either the result `00` or `11`. If the measurement outcome for one of the qubits is known, then the outcome for the other is fully determined.

### Exercise 1

Can you create the state $$\frac{1}{\sqrt{2}}\left(\vert01\rangle + \vert10\rangle\right)$$ using a similar procedure? There are multiple circuits to do so.

### Exercise 2

Can you create the state $$\frac{1}{\sqrt{2}}\left(\vert01\rangle - \vert10\rangle\right)$$ using a similar procedure? There are multiple circuits to do so; you may find the following single-qubit gate, called $Z$, useful: 
$$ Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}.$$

### Exercise 3

Can you create the state $$\frac{1}{\sqrt{2}}\left(\vert10\rangle - \vert01\rangle\right)$$ using a similar procedure? There are multiple circuits to do so.

How would you compare this state to the previous one?

## Measurements
In the above example, we simulated the action of a measurement by sampling counts from the statevector. A measurement can explicitly be inserted into a quantum circuit as well. Here is an example that creates the same Bell state and applies a measurement.

In [ ]:
mycircuit = QuantumCircuit(2, 2)
mycircuit.h(0)
mycircuit.cx(0,1)
mycircuit.measure([0,1], [0,1])
mycircuit.draw('mpl')

Two new features appeared in the circuit compared to our previous examples.

- First, note that we used a second argument in the `QuantumCircuit(2,2)` command. The second argument says that we will be creating a quantum circuit that contains two qubits (the first argument), and two classical bits (the second argument).
- Second, note that the `measure` command takes two arguments. The first argument is the set of qubits that will be measured. The second is the set of classical bits onto which the outcomes from the measurements of the qubits will be stored.

Since the above quantum circuit contains non-unitaries (the measurement gates), we will use `Qiskit`'s built-in basic simulator to run the circuit. To get the measurement counts, we can use the following code:

In [ ]:
from qiskit.providers.basic_provider import BasicSimulator
backend = BasicSimulator()
result = backend.run(mycircuit, shots=10000).result()
counts = result.get_counts(mycircuit)
plot_histogram(counts)

As you can see, the measurement outcomes are similar to when we sampled counts from the statevector itself.

# Quantum Fourier Transform

## QFT on 3 qubits
From here on, we show some more complex circuits. We can construct Quantum Fourier Transform circuit on 3 qubits as follows, see the lecture notes for reference. The reason we use `reverse_bits` is due to this:
[https://docs.quantum.ibm.com/build/bit-ordering](https://docs.quantum.ibm.com/build/bit-ordering)

In [ ]:
from math import pi
qc = QuantumCircuit(3)

qc.h(2)
qc.cp(pi/2, 1, 2)
qc.cp(pi/4, 0, 2)

qc.h(1)
qc.cp(pi/2, 0, 1) # CROT from qubit 0 to qubit 1
qc.h(0)

qc.swap(0,2)

display(qc.draw('mpl',reverse_bits=True))


## General QFT Function

Now, we are going to construct a universal circuit in Qiskit for the Quantum Fourier Transform (QFT).
To simplify the process, we will first build a circuit that applies the QFT with the qubits in reverse order and then perform a swap operation to rearrange them appropriately. Let's begin by designing the function that accurately rotates our qubits. Just like in the previous example with three qubits, we will start by correctly rotating the most significant qubit, which corresponds to the qubit with the highest index:

In [ ]:
def qft_circuit(circuit, num_qubits):

    for control in range(num_qubits - 1, -1, -1):

        # Apply Hadamard gates to control qubit
        circuit.h(control)

        # Apply controlled phase shift gates
        for target in range(control-1,-1,-1):
            pass

    # Perform the final swap operations
    for i in range(num_qubits // 2):
        pass

    return circuit

In [ ]:
# QFT
def qft_circuit(circuit, num_qubits):

    for control in range(num_qubits - 1, -1, -1):

        # Apply Hadamard gates to control qubit
        circuit.h(control)

        # Apply controlled phase shift gates
        for target in range(control-1,-1,-1):
            circuit.cp(2 * np.pi / 2**(control - target + 1), control, target)

    # Perform the final swap operations
    for i in range(num_qubits // 2):
        circuit.swap(i, num_qubits - i - 1)

    return circuit

In [ ]:
qc = QuantumCircuit(4)
qc = qft_circuit(qc, 4)
display(qc.draw('mpl',reverse_bits=True))

## Inverse QFT on first n qubits

In [ ]:
def inverse_qft(circuit, n):
    """Does the inverse QFT on the first n qubits in circuit"""

    # First we create a QFT circuit of the correct size:
    qft_circ = qft_circuit(QuantumCircuit(n), n)

    # Then we take the inverse of this circuit
    inv_qft_circ = qft_circ.inverse()

    # And add it to the first n qubits in our existing circuit
    circuit.append(inv_qft_circ, circuit.qubits[:n])

    return circuit.decompose() # .decompose() allows us to see the individual gates

# Exercise 4
We want to show that the following circuit leads to the quantum state obtained by applying QFT on state $\vert101\rangle$

In [ ]:
from math import pi
nqubits = 3
number = 5
qc = QuantumCircuit(nqubits)

qc.h([qubit for qubit in range(nqubits)])
qc.p(number*pi/4,0)
qc.p(number*pi/2,1)
qc.p(number*pi,2)

qc.draw('mpl',reverse_bits=True)


It is enough to apply inverse QFT on the quantum state. It should result in $\vert101\rangle$

In [ ]:
qc = inverse_qft(qc, nqubits)
qc.measure_all()
qc.draw('mpl',reverse_bits=True)

In [ ]:
from qiskit.providers.basic_provider import BasicSimulator
backend = BasicSimulator()
new_circuit = transpile(qc, backend)
result = backend.run(new_circuit, shots=1000).result()
counts = result.get_counts(qc)
plot_histogram(counts)



# Additional reading

- You can find more information about the QFT here: https://learn.qiskit.org/course/ch-algorithms/quantum-fourier-transform



# Grover search

- You can find more information about the QFT here: https://qiskit.org/documentation/tutorials/algorithms/06_grover.html

In [ ]:
#from qiskit.algorithms import AmplificationProblem
import qiskit_algorithms
from qiskit_algorithms import AmplificationProblem, Grover

# the state we desire to find is '11'
good_state = ['11']

# specify the oracle that marks the state '11' as a good solution
oracle = QuantumCircuit(2)
oracle.cz(0, 1)

# define Grover's algorithm
problem = AmplificationProblem(oracle, is_good_state=good_state)

# now we can have a look at the Grover operator that is used in running the algorithm
# (Algorithm circuits are wrapped in a gate to appear in composition as a block
# so we have to decompose() the op to see it expanded into its component gates.)
problem.grover_operator.decompose().draw(output='mpl',reverse_bits=True)

In [ ]:
from qiskit.primitives import Sampler


grover = Grover(sampler=Sampler())
result = grover.amplify(problem)
print('Result type:', type(result))
print()
print('Success!' if result.oracle_evaluation else 'Failure!')
print('Top measurement:', result.top_measurement)